In [313]:
###Importing Libraries
import pandas as pd
import numpy as np
import xgboost

from sklearn.model_selection import cross_val_score,KFold
from sklearn.cross_validation import  train_test_split
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from sklearn import grid_search

In [314]:
##Loading Training and Testing Data
train_data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")
train_data

,Name,Date of Birth,From,To,Flight Date,Flight Time,Booking Date,Class,Fare
0,Dr. FG61 FS88,1963-10-23,Mumbai,Kolkata,2016-11-04,13:50,2016-10-19,Business,14737.67
1,Dr. FG35 FS74,1981-10-09,Lucknow,Hyderabad,2016-09-19,18:30,2016-07-31,Economy,7621.71
2,Dr. MG45 MS99,1981-07-09,Lucknow,Patna,2016-05-27,10:50,2016-05-24,Economy,5655.81
3,Mr. MG93 MS13,1988-09-08,Mumbai,Lucknow,2016-07-24,14:40,2016-07-16,Economy,7436.94
4,Miss FG28 FS14,1994-11-18,Kolkata,Delhi,2016-12-21,10:20,2016-12-09,Economy,3282.54
5,Miss FG66 FS95,2005-11-08,Chennai,Patna,2016-05-17,14:10,2016-05-07,Business,22058.03
6,Mr. MG71 MS64,1948-09-11,Mumbai,Lucknow,2016-05-25,11:30,2016-05-20,Business,13321.88
7,Dr. MG32 MS6,1943-07-19,Chennai,Mumbai,2016-05-20,18:00,2016-04-26,Economy,5290.83
8,Mr. MG69 MS30,1972-05-15,Delhi,Lucknow,2016-11-27,12:30,2016-11-27,Economy,6402.71
9,Mr. MG14 MS53,1949-10-10,Delhi,Mumbai,2016-04-10,12:00,2016-04-09,Economy,7343.52


In [315]:
###Getting Labels for Class of Travel
train_data.Class = pd.Series( np.where( train_data.Class == 'Economy' , 0 , 1 ) , name = 'Class' )
test_data.Class = pd.Series( np.where( test_data.Class == 'Economy' , 0 , 1 ) , name = 'Class' )


###Mapping Destination and Source Cities to Numbers
mapping = {'Chennai':1  , 'Delhi':2, 'Hyderabad':3, 'Kolkata':4, 'Lucknow':5, 'Mumbai':6, 'Patna':7}
train_data=train_data.replace({'To': mapping, 'From': mapping})
test_data=test_data.replace({'To': mapping, 'From': mapping})

###Converting Flight Date, Booking Date to datetime and getting Age

train_data["Flight Date"] = pd.to_datetime(train_data["Flight Date"])
train_data["Date of Birth"] = pd.to_datetime(train_data["Date of Birth"])
train_data["Age"]=(train_data["Flight Date"]-train_data["Date of Birth"])/365.25
train_data["Age"]=train_data["Age"].dt.days

test_data["Flight Date"] = pd.to_datetime(test_data["Flight Date"])
test_data["Date of Birth"] = pd.to_datetime(test_data["Date of Birth"])
test_data["Age"]=(test_data["Flight Date"]-test_data["Date of Birth"])/365.25
test_data["Age"]=test_data["Age"].dt.days

test_data["Booking Date"] = pd.to_datetime(test_data["Booking Date"])
train_data["Booking Date"] = pd.to_datetime(train_data["Booking Date"])


###Getting the weekday from Flight Date and Booking Date
train_data['Flight Day'] = train_data['Flight Date'].apply(lambda x: x.weekday())
test_data['Flight Day'] = test_data['Flight Date'].apply(lambda x: x.weekday())

train_data['Booking Day'] = train_data['Booking Date'].apply(lambda x: x.weekday())
test_data['Booking Day'] = test_data['Booking Date'].apply(lambda x: x.weekday())

###Dividing the Weekday into Two Categories
#bins = [-1, 3, 7]
#names = [0, 1]

#train_data['WeekDay']=pd.cut(train_data['Flight Day'], bins, labels=names)
#test_data['WeekDay']=pd.cut(test_data['Flight Day'], bins, labels=names)
#train_data['WeekDay']=pd.to_numeric(train_data['WeekDay'])
#test_data['WeekDay']=pd.to_numeric(test_data['WeekDay'])



##Removing Name, DateofBirth after extracting Gender and Two Numbers
### Adding Gender to Data
X_trn = []
for i in train_data['Name']:
    c1 = i.split(' ')[1][0]
    if(c1 is 'M'):
        X_trn.append(1)
    else:
        X_trn.append(0)

X_tst = []
for i in test_data['Name']:
    c1 = i.split(' ')[1][0]
    if(c1 is 'M'):
        X_tst.append(1)
    else:
        X_tst.append(0)


train_data['Gender'] = X_trn
test_data['Gender'] = X_tst


#train_data=train_data.drop(columns=['Name', 'Date of Birth'])
#test_data=test_data.drop(columns=['Name', 'Date of Birth'])


###Getting the month of booking and travel
train_data["Flight Month"] = train_data["Flight Date"].dt.month
train_data["Booking Month"] = train_data["Booking Date"].dt.month

test_data["Flight Month"] = test_data["Flight Date"].dt.month
test_data["Booking Month"] = test_data["Booking Date"].dt.month

###Getting the Weekday of booking and travel
train_data["Flight Week"] = train_data["Flight Date"].dt.week
train_data["Booking Week"] = train_data["Booking Date"].dt.week

test_data["Flight Week"] = test_data["Flight Date"].dt.week
test_data["Booking Week"] = test_data["Booking Date"].dt.week



###Finding the travel time
train_data['Flight Time'] = train_data['Flight Time'].str.extract('(\d\d)',expand=True)
test_data['Flight Time'] = test_data['Flight Time'].str.extract('(\d\d)',expand=True)


train_data['Flight Time']=pd.to_numeric(train_data['Flight Time'])
test_data['Flight Time']=pd.to_numeric(test_data['Flight Time'])



#### Getting the Day diff between the booking of Flight and travel
train_data['Diff']=(train_data['Flight Date']-train_data['Booking Date'])
test_data['Diff']=(test_data['Flight Date']-test_data['Booking Date'])
train_data['Diff']=train_data['Diff'].dt.days
test_data['Diff']=test_data['Diff'].dt.days


###Dividing the Age into Brackets
bins = [0, 5, 18, 35, 65, 100]
names = [0, 1, 2, 3, 4]

train_data['AgeRange'] = pd.cut(train_data['Age'], bins, labels=names)
test_data['AgeRange'] = pd.cut(test_data['Age'], bins, labels=names)

train_data=train_data.drop(columns=['Age'])
test_data=test_data.drop(columns=['Age'])



#####
train_data["AgeRange"]=pd.to_numeric(train_data["AgeRange"])
test_data["AgeRange"]=pd.to_numeric(test_data["AgeRange"])





#### Weekend Feature 
X = []
for i in train_data['Flight Day']:
    if(i==6 or i==5):
        X.append(1)
    else:
        X.append(0)
        
train_data['Weekend'] = X

X = []
for i in test_data['Flight Day']:
    if(i==6 or i==5):
        X.append(1)
    else:
        X.append(0)
        
test_data['Weekend'] = X

### Peak Time Feature 

X = []
for i in train_data['Flight Time']:
    if(i>=18 and i<=23):
        X.append(5)
    elif(i>=12 and i<18):
        X.append(3)
    else:
        X.append(1)
        
train_data['PeakTime'] = X

X = []
for i in test_data['Flight Time']:
    if(i>=18 and i<=23):
        X.append(5)
    elif(i>=12 and i<18):
        X.append(3)
    else:
        X.append(1)

test_data['PeakTime'] = X

### holidays feature 

holidays = ['2016-01-01','2016-03-07','2016-04-15','2016-05-21','2016-08-15','2016-09-05','2016-10-02','2016-10-30','2016-01-14','2016-03-24','2016-04-20','2016-07-06','2016-08-18','2016-09-12','2016-10-11','2016-12-13','2016-01-26','2016-03-25','2016-05-01','2016-07-06','2016-08-25','2016-09-14','2016-10-12','2016-12-25']
df = pd.to_datetime(holidays)
dr = train_data['Flight Date']

X = []
for i in dr:
    if(i in df):
        X.append(1)
    else:
        X.append(0)

train_data['holidays'] = X

dr = test_data['Flight Date']

X = []
for i in dr:
    if(i in df):
        X.append(1)
    else:
        X.append(0)

test_data['holidays'] = X




















In [316]:
train_data

,Name,Date of Birth,From,To,Flight Date,Flight Time,Booking Date,Class,Fare,Flight Day,...,FromD,FromM,FromL,ToC,ToK,ToH,ToP,ToD,ToM,ToL
0,Dr. FG61 FS88,1963-10-23,6,4,2016-11-04,13,2016-10-19,1,14737.67,4,...,0,0,0,0,0,0,0,0,0,0
1,Dr. FG35 FS74,1981-10-09,5,3,2016-09-19,18,2016-07-31,0,7621.71,0,...,0,0,0,0,0,0,0,0,0,0
2,Dr. MG45 MS99,1981-07-09,5,7,2016-05-27,10,2016-05-24,0,5655.81,4,...,0,0,0,0,0,0,0,0,0,0
3,Mr. MG93 MS13,1988-09-08,6,5,2016-07-24,14,2016-07-16,0,7436.94,6,...,0,0,0,0,0,0,0,0,0,0
4,Miss FG28 FS14,1994-11-18,4,2,2016-12-21,10,2016-12-09,0,3282.54,2,...,0,0,0,0,0,0,0,0,0,0
5,Miss FG66 FS95,2005-11-08,1,7,2016-05-17,14,2016-05-07,1,22058.03,1,...,0,0,0,0,0,0,0,0,0,0
6,Mr. MG71 MS64,1948-09-11,6,5,2016-05-25,11,2016-05-20,1,13321.88,2,...,0,0,0,0,0,0,0,0,0,0
7,Dr. MG32 MS6,1943-07-19,1,6,2016-05-20,18,2016-04-26,0,5290.83,4,...,0,0,0,0,0,0,0,0,0,0
8,Mr. MG69 MS30,1972-05-15,2,5,2016-11-27,12,2016-11-27,0,6402.71,6,...,0,0,0,0,0,0,0,0,0,0
9,Mr. MG14 MS53,1949-10-10,2,6,2016-04-10,12,2016-04-09,0,7343.52,6,...,0,0,0,0,0,0,0,0,0,0


In [317]:

###Getting the X and Y values
X = train_data[['From', 'To', 'Gender','Flight Week','Class','Diff','AgeRange' ,'PeakTime', 'Weekend','holidays']]
y=train_data[['Fare']]
test_data=test_data[['From', 'To','Gender','Flight Week','Class','Diff','AgeRange','PeakTime', 'Weekend','holidays']]
###Data is prepared
####Train test Split
X_train, X_Val, y_train, Y_Val = train_test_split(X, y)


In [318]:
###Basic Linear Regression
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
print(r2_score(Y_Val, reg.predict(X_Val))) 

0.4914744825107158


In [319]:
####Lasso Regression
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.5)
clf.fit(X_train,y_train)
print(r2_score(Y_Val, clf.predict(X_Val))) 

0.49150362989395346


In [312]:
####XG Boost
xgb = xgboost.XGBRegressor(n_estimators=750, learning_rate=0.09, gamma=0.05, subsample=0.85,
                           colsample_bytree=1, max_depth=4)

xgb.fit(X,y)
#print(xgb.score(X_Val,Y_Val))
y_test_xgb=xgb.predict(test_data)
prediction = pd.DataFrame(y_test_xgb, columns=['predictions']).to_csv('XGBoostFinal_hyper_2.csv')

In [274]:
#####Grid Search GBM
from sklearn.ensemble import GradientBoostingRegressor 

parameters={'min_impurity_split':[1e-1],'learning_rate':[1e-1],'min_samples_split':[7],'verbose':[2],'max_depth':[7],'min_samples_leaf':[1],'subsample':[1.0],'loss':['ls'],'n_estimators':[100]}
gbm = grid_search.GridSearchCV(GradientBoostingRegressor(),parameters) 
gbm.fit(X_train,y_train)
print(r2_score(Y_Val, gbm.predict(X_Val))) 
#y_test_gbm=gbm.predict(test_data)
#prediction = pd.DataFrame(y_test_gbm, columns=['predictions']).to_csv('GBMFinal.csv')


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/pyth

      Iter       Train Loss   Remaining Time 
         1    38412129.3367            0.73s
         2    33102327.2030            0.73s
         3    28783055.9371            0.73s
         4    25233381.3875            0.72s
         5    22260204.9662            0.77s
         6    19704897.8275            0.77s
         7    17688113.2732            0.76s
         8    16009416.7769            0.74s
         9    14541976.4151            0.73s
        10    13356801.2095            0.71s
        11    12276362.2041            0.70s
        12    11402110.6023            0.69s
        13    10652068.5925            0.68s
        14     9982839.2968            0.67s
        15     9444907.3183            0.65s
        16     8891483.3392            0.64s
        17     8476740.7329            0.64s
        18     8113057.8143            0.63s
        19     7732410.5421            0.62s
        20     7335405.4816            0.61s
        21     7063451.2020            0.60s
        2

/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/s

        27     5346402.7378            0.57s
        28     5120290.4790            0.57s
        29     4918322.8983            0.57s
        30     4734277.6097            0.56s
        31     4570637.2294            0.55s
        32     4422403.4669            0.54s
        33     4234141.1637            0.54s
        34     4140178.1929            0.53s
        35     4021786.2975            0.52s
        36     3931663.2917            0.51s
        37     3825668.9009            0.50s
        38     3696960.3280            0.49s
        39     3597966.7585            0.48s
        40     3499604.0381            0.47s
        41     3413200.0947            0.46s
        42     3345840.9877            0.45s
        43     3283823.5722            0.45s
        44     3109456.0755            0.44s
        45     3018920.1341            0.43s
        46     2971275.0706            0.42s
        47     2829017.5688            0.42s
        48     2739472.9752            0.41s
        49

/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/s

...

/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/s

        50     3373592.2936            0.58s
        51     3280696.7665            0.57s
        52     3233737.4394            0.55s
        53     3156876.6235            0.54s
        54     3113766.5447            0.53s
        55     3078734.5710            0.52s
        56     3043306.1786            0.51s
        57     3011364.4662            0.50s
        58     2942886.8002            0.48s
        59     2889767.5322            0.47s
        60     2861800.4436            0.46s
        61     2833402.4897            0.45s
        62     2804068.1325            0.44s
        63     2786625.6362            0.42s
        64     2711848.0088            0.41s
        65     2679562.1081            0.40s
        66     2610156.5547            0.39s
        67     2557031.1746            0.38s


/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/s

        68     2531005.8029            0.37s
        69     2506553.0381            0.36s
        70     2459867.2668            0.35s
        71     2419045.7549            0.34s
        72     2409512.8579            0.32s
        73     2376171.7319            0.31s
        74     2364212.1282            0.30s
        75     2330321.5108            0.29s
        76     2315047.9028            0.28s
        77     2297825.3964            0.26s
        78     2262365.1255            0.25s
        79     2238578.2189            0.24s
        80     2221091.4700            0.23s
        81     2202398.1840            0.22s
        82     2181730.4701            0.21s
        83     2170702.8411            0.19s
        84     2161588.8706            0.18s
        85     2134350.4437            0.17s
        86     2104899.4283            0.16s
        87     2085117.0442            0.15s
        88     2065380.1514            0.14s
        89     2041207.2311            0.13s
        90

/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/s

In [275]:
print(r2_score(Y_Val, gbm.predict(X_Val))) 


0.7914910725925524


In [276]:
from sklearn.neural_network import MLPRegressor
mlp=MLPRegressor(hidden_layer_sizes=(500, 150 ), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
mlp.fit(X_train, y_train)
print(r2_score(Y_Val, mlp.predict(X_Val))) 

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5754365879175065


In [278]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=100, random_state=0,n_estimators=500)
regr.fit(X_train, y_train)
print(r2_score(Y_Val, regr.predict(X_Val))) 


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.729166903537898


In [320]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBRegressor(n_estimators=100, learning_rate=0.1, gamma=0.5, subsample=0.75,
                           colsample_bytree=0.7, max_depth=7), param_grid = param_test1,n_jobs=4,iid=False, cv=5)
gsearch1.fit(X,y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.58655, std: 0.02381, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.58636, std: 0.02385, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.58653, std: 0.02320, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.57737, std: 0.02652, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.57851, std: 0.02861, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.57808, std: 0.02905, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.55272, std: 0.02933, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.55375, std: 0.03081, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.55582, std: 0.03116, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.52586, std: 0.02873, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.53309, std: 0.03324, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.53695, std: 0.03065, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 1